In [1]:
pip install quickdraw

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Loading data from quickdraw API

In [3]:
#Data loading & processing
import os
import datetime
from quickdraw import QuickDrawData, QuickDrawDataGroup
from pathlib import Path  # Import the Path class from pathlib
from tensorflow.keras.utils import image_dataset_from_directory
import matplotlib.pyplot as plt

#Model
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.layers import Rescaling, BatchNormalization, Flatten, Dense, Dropout


In [ ]:
image_size = (75, 75)

def generate_class_images(name, max_drawings, recognized):
    directory = Path("dataset/" + name)

    if not directory.exists():
        directory.mkdir(parents=True)

    images = QuickDrawDataGroup(name, max_drawings=max_drawings, recognized=recognized)
    for img in images.drawings:
        filename = directory.as_posix() + "/" + str(img.key_id) + ".png"
        img.get_image(stroke_width=3).resize(image_size).save(filename)

for label in QuickDrawData().drawing_names:
    generate_class_images(label, max_drawings=1200, recognized=True)

loading aircraft carrier drawings
load complete
loading airplane drawings
load complete
loading alarm clock drawings
load complete
loading ambulance drawings
load complete
loading angel drawings
load complete
loading animal migration drawings
load complete
loading ant drawings
load complete
loading anvil drawings
load complete
loading apple drawings
load complete
loading arm drawings
load complete
loading asparagus drawings
load complete
loading axe drawings
load complete
loading backpack drawings
load complete
loading banana drawings
load complete
loading bandage drawings
load complete
loading barn drawings
load complete
loading baseball bat drawings
load complete
loading baseball drawings
load complete
loading basket drawings
load complete
loading basketball drawings
load complete
loading bat drawings
load complete
loading bathtub drawings
load complete
loading beach drawings
load complete
loading bear drawings
load complete
loading beard drawings
load complete
loading bed drawings
l

In [5]:
batch_size = 32
image_size = (75, 75)

dataset_dir = "/content/dataset"


train_ds = image_dataset_from_directory(
    dataset_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    color_mode="grayscale",
    image_size=image_size,
    batch_size=batch_size
)

val_ds = image_dataset_from_directory(
    dataset_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    color_mode="grayscale",
    image_size=image_size,
    batch_size=batch_size
)

Found 414000 files belonging to 345 classes.
Using 331200 files for training.
Found 414000 files belonging to 345 classes.
Using 82800 files for validation.


In [9]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define image size
image_size = (75, 75)

# Preprocess the data
def preprocess(image, label):
    image = tf.image.resize(image, image_size)  # Resize images to the desired size
    image = tf.image.convert_image_dtype(image, tf.float32)  # Convert to float32
    image = preprocess_input(image)  # Preprocess according to InceptionV3 requirements

    return image, label

# Apply preprocessing to the datasets
train_ds = train_ds.map(preprocess)
val_ds = val_ds.map(preprocess)

In [12]:
plt.figure(figsize=(8, 8))
#for images, labels in train_ds.take(1):
#    for i in range(9):
#        ax = plt.subplot(3, 3, i + 1)
#        data = images[i].numpy().astype("uint8")
#        plt.imshow(data, cmap='gray', vmin=0, vmax=255)
#        plt.title(train_ds.class_names[labels[i]])
#        plt.axis("off")

<Figure size 800x800 with 0 Axes>

<Figure size 800x800 with 0 Axes>

In [ ]:
# Define the model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.GlobalAveragePooling2D(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(345, activation='softmax')  # Adjust the output layer based on your number of classes
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 13, 13, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 5, 5, 64)          0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 3, 3, 128)         73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 1, 1, 128)         0

In [ ]:
# Train the model
history = model.fit(train_ds, validation_data=val_ds, epochs=10)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(val_ds)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Epoch 1/10
10350/10350 [==============================] - 455s 44ms/step - loss: 5.8489 - accuracy: 0.0028 - val_loss: 5.8448 - val_accuracy: 0.0026
Epoch 2/10
10350/10350 [==============================] - 421s 41ms/step - loss: 5.8447 - accuracy: 0.0027 - val_loss: 5.8449 - val_accuracy: 0.0026
Epoch 3/10
10350/10350 [==============================] - 406s 39ms/step - loss: 5.8447 - accuracy: 0.0028 - val_loss: 5.8449 - val_accuracy: 0.0026
Epoch 4/10
10350/10350 [==============================] - 410s 40ms/step - loss: 5.8447 - accuracy: 0.0027 - val_loss: 5.8449 - val_accuracy: 0.0026
Epoch 5/10
10350/10350 [==============================] - 413s 40ms/step - loss: 5.8447 - accuracy: 0.0027 - val_loss: 5.8449 - val_accuracy: 0.0026
Epoch 6/10
10350/10350 [==============================] - 414s 40ms/step - loss: 5.8447 - accuracy: 0.0027 - val_loss: 5.8449 - val_accuracy: 0.0026
Epoch 7/10
10350/10350 [==============================] - 409s 40ms/step - loss: 5.8447 - accuracy: 0.0027

In [12]:
n_classes = 345
input_shape = (75, 75, 1)

model1 = models.Sequential([
    Rescaling(1. / 255, input_shape=input_shape),
    BatchNormalization(),

    layers.Conv2D(6, kernel_size=(3, 3), padding="same", activation="relu"),
    layers.Conv2D(8, kernel_size=(3, 3), padding="same", activation="relu"),
    layers.Conv2D(10, kernel_size=(3, 3), padding="same", activation="relu"),
    BatchNormalization(),
    layers.MaxPooling2D(pool_size=(2, 2)),

    Flatten(),

    Dense(700, activation="relu"),
    BatchNormalization(),
    Dropout(0.2),

    Dense(500, activation="relu"),
    BatchNormalization(),
    Dropout(0.2),

    Dense(400, activation="relu"),
    Dropout(0.2),

    Dense(n_classes, activation="softmax")
])

# Compile the model with a lower learning rate
model1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model1.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_3 (Rescaling)     (None, 75, 75, 1)         0         
                                                                 
 batch_normalization_12 (Ba  (None, 75, 75, 1)         4         
 tchNormalization)                                               
                                                                 
 conv2d_9 (Conv2D)           (None, 75, 75, 6)         60        
                                                                 
 conv2d_10 (Conv2D)          (None, 75, 75, 8)         440       
                                                                 
 conv2d_11 (Conv2D)          (None, 75, 75, 10)        730       
                                                                 
 batch_normalization_13 (Ba  (None, 75, 75, 10)        40        
 tchNormalization)                                    

In [13]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = TensorBoard(logdir, histogram_freq=1)

epochs = 10

model1.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
    verbose=1,
    callbacks=[tensorboard_callback]
)

model.save('./models/model_' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

Epoch 1/10
 3971/10350 [==========>...................] - ETA: 59:34 - loss: 3.4249 - accuracy: 0.2498

KeyboardInterrupt: 

In [14]:
model = models.Sequential([
    Rescaling(1. / 255, input_shape=input_shape),
    layers.Conv2D(16, kernel_size=(3, 3), activation="relu"),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.Dense(n_classes, activation="softmax")
])

# Compile the model with a lower learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_4 (Rescaling)     (None, 75, 75, 1)         0         
                                                                 
 conv2d_12 (Conv2D)          (None, 73, 73, 16)        160       
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 36, 36, 16)        0         
 g2D)                                                            
                                                                 
 conv2d_13 (Conv2D)          (None, 34, 34, 32)        4640      
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 17, 17, 32)        0         
 g2D)                                                            
                                                                 
 flatten_4 (Flatten)         (None, 9248)             

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
#using model.save_weights() to save the weights of the model in HDF5 format

In [16]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = TensorBoard(logdir, histogram_freq=1)

epochs = 10

model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
    verbose=1,
    callbacks=[tensorboard_callback]
)

model.save('/content/drive/model_' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

Epoch 1/10
10350/10350 [==============================] - 1530s 148ms/step - loss: 3.0657 - accuracy: 0.3251 - val_loss: 2.4864 - val_accuracy: 0.4128
Epoch 2/10
10350/10350 [==============================] - 1540s 149ms/step - loss: 2.3056 - accuracy: 0.4482 - val_loss: 2.2612 - val_accuracy: 0.4569
Epoch 3/10
10350/10350 [==============================] - 1557s 150ms/step - loss: 2.0709 - accuracy: 0.4942 - val_loss: 2.1704 - val_accuracy: 0.4789
Epoch 4/10
10350/10350 [==============================] - 1544s 149ms/step - loss: 1.9263 - accuracy: 0.5231 - val_loss: 2.1548 - val_accuracy: 0.4815
Epoch 5/10
10350/10350 [==============================] - 1546s 149ms/step - loss: 1.8140 - accuracy: 0.5450 - val_loss: 2.1851 - val_accuracy: 0.4827
Epoch 6/10
10350/10350 [==============================] - 1534s 148ms/step - loss: 1.7251 - accuracy: 0.5643 - val_loss: 2.1782 - val_accuracy: 0.4831
Epoch 7/10
10350/10350 [==============================] - 1507s 146ms/step - loss: 1.6436 - ac